# Data cleaning

## Imports and extensions

We will firstly load the required extensions and import libraries:

In [1]:
# Autoreloading modules
%load_ext autoreload
%autoreload 2

In [2]:
import os

import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine

## Connecting to DB

Next step is to load database connection parameters from environmental variables and then use them to establish the connection.

In [3]:
load_dotenv()

True

In [4]:
dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')
dst_db = os.environ.get('DB_DESTINATION_NAME')

In [5]:
dst_conn = create_engine(
    f'postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}'
    )

## Pulling a table from DB

Now that the connection has been established, we can load the table, which we have obtained by firstly running `flats_init_etl` pipeline, into a dataframe:

In [6]:
try:
    data = pd.read_sql(f'SELECT * FROM flats_init', dst_conn)
except:
    print(f"Relation does not exist.")
else:
    display(data.head())

,id,flat_id,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator,floor,kitchen_area,living_area,rooms,is_apartment,studio,total_area,price
0,1,0,1965,6,55.717113,37.781120,2.64,84,12,True,9,9.9,19.900000,1,False,False,35.099998,9500000
1,2,1,2001,2,55.794849,37.608013,3.00,97,10,True,7,0.0,16.600000,1,False,False,43.000000,13500000
2,3,2,2000,4,55.740040,37.761742,2.70,80,10,True,9,9.0,32.000000,2,False,False,56.000000,13500000
3,4,3,2002,4,55.672016,37.570877,2.64,771,17,True,1,10.1,43.099998,3,False,False,76.000000,20000000
4,5,4,1971,1,55.808807,37.707306,2.60,208,9,True,3,3.0,14.000000,1,False,False,24.000000,5200000


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141362 entries, 0 to 141361
Data columns (total 18 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 141362 non-null  int64  
 1   flat_id            141362 non-null  int64  
 2   build_year         141362 non-null  int64  
 3   building_type_int  141362 non-null  int64  
 4   latitude           141362 non-null  float64
 5   longitude          141362 non-null  float64
 6   ceiling_height     141362 non-null  float64
 7   flats_count        141362 non-null  int64  
 8   floors_total       141362 non-null  int64  
 9   has_elevator       141362 non-null  bool   
 10  floor              141362 non-null  int64  
 11  kitchen_area       141362 non-null  float64
 12  living_area        141362 non-null  float64
 13  rooms              141362 non-null  int64  
 14  is_apartment       141362 non-null  bool   
 15  studio             141362 non-null  bool   
 16  to

In [8]:
data.describe().T.round(2)

,count,mean,std,min,25%,50%,75%,max
id,141362.0,70681.50,40807.84,1.00,35341.25,70681.50,106021.75,1.413620e+05
flat_id,141362.0,70680.50,40807.84,0.00,35340.25,70680.50,106020.75,1.413610e+05
build_year,141362.0,1986.60,22.14,1901.00,1969.00,1985.00,2007.00,2.023000e+03
building_type_int,141362.0,3.23,1.46,0.00,2.00,4.00,4.00,6.000000e+00
latitude,141362.0,55.73,0.10,55.21,55.65,55.72,55.81,5.601000e+01
longitude,141362.0,37.59,0.15,36.86,37.49,37.58,37.69,3.795000e+01
ceiling_height,141362.0,2.75,0.22,2.00,2.64,2.64,2.80,2.700000e+01
flats_count,141362.0,251.99,207.34,1.00,111.00,200.00,324.00,4.455000e+03
floors_total,141362.0,14.11,6.90,1.00,9.00,14.00,17.00,9.900000e+01
floor,141362.0,7.47,5.72,1.00,3.00,6.00,10.00,5.600000e+01


The above information enables us to conclude that there are no missing values, though overly high and overly low values of the flat price seem to be dubious. The same can be said about a very small value of the minimal ceiling height. This indicates the presence of outliers in the data.

## Preprocessing

We will consider the loaded data in three directions:

* Duplicates
* Missing values
* Outliers

Having got rid of the above problems, we will be able to build a cleaned dataset which can then be used for model training.

### Duplicates

In [9]:
feature_cols = data.columns.drop(['id', 'flat_id']).tolist()
is_duplicated_features = data.duplicated(subset=feature_cols, keep=False)

data[is_duplicated_features].sort_values(feature_cols)

,id,flat_id,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator,floor,kitchen_area,living_area,rooms,is_apartment,studio,total_area,price
52640,52641,52640,1901,1,55.759594,37.646488,3.2,26,4,False,3,0.0,0.0,4,False,False,120.000000,60000000
54009,54010,54009,1901,1,55.759594,37.646488,3.2,26,4,False,3,0.0,0.0,4,False,False,120.000000,60000000
90882,90883,90882,1902,1,55.748661,37.656487,3.0,32,4,False,4,11.0,48.0,2,False,False,63.000000,28000000
122183,122184,122183,1902,1,55.748661,37.656487,3.0,32,4,False,4,11.0,48.0,2,False,False,63.000000,28000000
63568,63569,63568,1902,1,55.754581,37.594685,3.2,15,6,True,5,10.0,74.0,3,False,False,100.000000,50000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100793,100794,100793,2022,2,55.835766,37.491871,2.8,264,22,True,2,18.0,0.0,3,False,False,49.400002,18000000
100937,100938,100937,2022,2,55.835766,37.491871,2.8,264,22,True,2,18.0,0.0,3,False,False,49.400002,18000000
132927,132928,132927,2022,2,55.835766,37.491871,2.8,264,22,True,2,18.0,0.0,3,False,False,49.400002,18000000
32684,32685,32684,2022,2,55.835766,37.491871,2.8,264,22,True,14,0.0,0.0,1,False,False,36.000000,8100000


In [10]:
data.duplicated(subset=feature_cols, keep=False).sum()

17425

In [11]:
def remove_duplicates(data):
    """Removes duplicates from the data."""
    feature_cols = data.columns.drop(['id', 'flat_id']).tolist()
    is_duplicated_features = data.duplicated(subset=feature_cols, keep=False)
    data = data[~is_duplicated_features].reset_index(drop=True)

    return data

In [12]:
data = remove_duplicates(data)

In [13]:
data.duplicated(subset=feature_cols, keep=False).sum()

0

### Missing values

In [14]:
data.isnull().sum()

id                   0
flat_id              0
build_year           0
building_type_int    0
latitude             0
longitude            0
ceiling_height       0
flats_count          0
floors_total         0
has_elevator         0
floor                0
kitchen_area         0
living_area          0
rooms                0
is_apartment         0
studio               0
total_area           0
price                0
dtype: int64

### Outliers

In [15]:
num_cols = data[feature_cols].select_dtypes(['float', 'int']).columns
threshold = 1.5
potential_outliers = pd.DataFrame()

for col in num_cols:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    margin = threshold * IQR
    lower = Q1 - margin
    upper = Q3 + margin
    potential_outliers[col] = ~data[col].between(lower, upper)

outliers = potential_outliers.any(axis=1)

In [16]:
data[outliers]

,id,flat_id,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator,floor,kitchen_area,living_area,rooms,is_apartment,studio,total_area,price
3,4,3,2002,4,55.672016,37.570877,2.64,771,17,True,1,10.1,43.099998,3,False,False,76.000000,20000000
31,35,34,1992,4,55.983387,37.152309,2.64,379,14,True,8,8.5,19.000000,1,False,False,38.000000,5800000
36,42,41,2011,2,55.676590,37.634090,3.00,625,24,True,24,14.0,20.000000,1,False,False,58.000000,15990000
39,45,44,2017,2,55.627728,37.342995,3.00,646,9,True,9,0.0,0.000000,1,False,False,40.599998,6942000
42,49,48,2004,2,55.703480,37.511040,3.00,284,17,True,8,14.0,64.800003,3,False,False,93.300003,37000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123918,141339,141338,2010,2,55.715485,37.479034,3.00,183,24,True,24,0.0,110.000000,5,False,False,184.000000,92000000
123920,141341,141340,2012,2,55.744308,37.419521,3.00,332,40,True,39,10.0,80.000000,2,False,False,112.000000,45000000
123923,141345,141344,2005,2,55.681267,37.582401,2.80,107,17,True,14,17.1,80.199997,3,False,False,136.000000,30000000
123924,141346,141345,2013,4,55.683201,37.648277,2.80,524,20,True,12,12.0,53.799999,3,False,False,97.239998,30990000


In [17]:
def remove_outliers(data):
    """Removes outliers from the data."""
    num_cols = data[feature_cols].select_dtypes(['float', 'int']).columns
    threshold = 1.5
    potential_outliers = pd.DataFrame()

    for col in num_cols:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        margin = threshold * IQR
        lower = Q1 - margin
        upper = Q3 + margin
        potential_outliers[col] = ~data[col].between(lower, upper)

    outliers = potential_outliers.any(axis=1)
    data = data[~outliers]

    return data

In [18]:
data = remove_outliers(data)

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 97178 entries, 0 to 123936
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 97178 non-null  int64  
 1   flat_id            97178 non-null  int64  
 2   build_year         97178 non-null  int64  
 3   building_type_int  97178 non-null  int64  
 4   latitude           97178 non-null  float64
 5   longitude          97178 non-null  float64
 6   ceiling_height     97178 non-null  float64
 7   flats_count        97178 non-null  int64  
 8   floors_total       97178 non-null  int64  
 9   has_elevator       97178 non-null  bool   
 10  floor              97178 non-null  int64  
 11  kitchen_area       97178 non-null  float64
 12  living_area        97178 non-null  float64
 13  rooms              97178 non-null  int64  
 14  is_apartment       97178 non-null  bool   
 15  studio             97178 non-null  bool   
 16  total_area         97178 n

In [20]:
dst_conn.dispose()